In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [3]:
def train(model, n_epochs, trainloader, optimizer, lossfn, print_iter=None):
    
    iter = 0
    train_iters = len(trainloader)
    if print_iter is None:
        print_iter = train_iters//4
    model.train()
    for epoch in range(n_epochs):
        for x, y in trainloader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = lossfn(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if iter%print_iter == 0:
                print(f'{epoch}:{iter} || loss: {loss.item():.4f}')
            # print(f'{epoch}:{iter} || loss: {loss.item():.4f}')
            iter+=1
        print(f'Epoch done::{epoch}:{iter} || loss: {loss.item():.4f}')

In [4]:
@torch.no_grad()
def evaluate(testloader, model):
    model.eval()
    totalcnt = 0
    correctcnt = 0
    for x,y in testloader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        yhat = logits.argmax(1)
        totalcnt += yhat.shape[0]
        correctcnt += (yhat == y).sum().item()
    print(totalcnt, correctcnt)
    return correctcnt/totalcnt

# Lenet

In [5]:
# Define transformations for the training and test sets
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST datasets
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False)


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride = 1)
        self.batch1 = nn.BatchNorm2d(6)
        self.maxpool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.batch2 = nn.BatchNorm2d(16)
        self.maxpool2 = nn.MaxPool2d(2)

        self.l1 = nn.Linear(16*4*4, 120)
        self.l2 = nn.Linear(120, 84)
        self.l3 = nn.Linear(84, 10)

        self.relu = nn.ReLU()


    # defining the network flow
    def forward(self, x):
        out = self.relu(self.batch1(self.conv1(x)))
        out = self.maxpool1(out)

        out = self.relu(self.batch2(self.conv2(out)))
        out = self.maxpool2(out)

        out = out.view(out.shape[0], -1)

        out = self.relu(self.l1(out))
        out = self.relu(self.l2(out))
        out = self.l3(out)

        return out



lenet = LeNet()
lenet.to(device);

optimizer = torch.optim.Adam(lenet.parameters(), lr=0.01)
lossfn = nn.CrossEntropyLoss()

n_epochs = 3

train(lenet, 2, trainloader, optimizer, lossfn)

print(evaluate(trainloader, lenet))

print(evaluate(testloader, lenet))



# AlexNet

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227,227), antialias=True),
    transforms.Normalize((0.5,), (0.5,))
    ])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, 128, True)
testloader = DataLoader(testset, 128, False)



class AlexNet(nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 96, 11, 4, 0) #55
    self.bn1 = nn.BatchNorm2d(96) # 27
    self.conv2 = nn.Conv2d(96, 256, 5, 1, 2) # 27
    self.bn2 = nn.BatchNorm2d(256) # 13
    self.conv3 = nn.Conv2d(256, 384, 3, 1, 1) # 13
    self.bn3 = nn.BatchNorm2d(384) # 13
    self.conv4 = nn.Conv2d(384, 384, 3, 1, 1) # 13
    self.bn4 = nn.BatchNorm2d(384) # 13
    self.conv5 = nn.Conv2d(384, 256, 3, 1, 1) # 13
    self.bn5 = nn.BatchNorm2d(256) # 6
    self.fc1 = nn.Linear(6*6*256, 4096)
    self.fc2 = nn.Linear(4096, 4096)
    self.fc3 = nn.Linear(4096, 10)

    self.pool = nn.MaxPool2d((3,3), stride=2)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)

  def forward(self, x):
    x = self.pool(self.relu(self.bn1(self.conv1(x))))
    x = self.pool(self.relu(self.bn2(self.conv2(x))))
    x = self.relu(self.bn3(self.conv3(x)))
    x = self.relu(self.bn4(self.conv4(x)))
    x = self.pool(self.relu(self.bn5(self.conv5(x))))
    x = x.view(x.shape[0], -1)
    x = self.relu(self.fc1(self.dropout(x)))
    x = self.relu(self.fc2(self.dropout(x)))
    x = self.fc3(x)
    return x


alexnet = AlexNet()
alexnet.to(device);



optimizer = torch.optim.AdamW(alexnet.parameters(), lr = 0.005)
lossfn = nn.CrossEntropyLoss()

train(alexnet, 50, trainloader, optimizer, lossfn)

print(evaluate(testloader, alexnet))

evaluate(trainloader, alexnet)


# torch.save(alexnet.state_dict(), 'alexnetv1.pt')

# ResNet

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224), antialias=True),
    transforms.Normalize((0.5,), (0.5,))
    ])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, 128, True)
testloader = DataLoader(testset, 128, False)


In [8]:
class ResBlock(nn.Module):
    def __init__(self, inp, out, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inp, out, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out)
        self.conv2 = nn.Conv2d(out, out, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out)
        self.downsample = downsample
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample is not None:
            x = self.downsample(x)
        out = self.relu(out + x)
        return out
    

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(3, 2);
        
        self.l1 = self.make_layer(3, 64, 1)
        self.l2 = self.make_layer(4, 128, 2)
        self.l3 = self.make_layer(6, 256, 2)
        self.l4 = self.make_layer(3, 512, 2)
        
        self.avgpool = nn.AvgPool2d(7);
        self.fc = nn.Linear(512, 10)
        
        self.relu = nn.ReLU()
        
    def make_layer(self,num_layer, out, stride):
        downsample = None
        if stride != 1 or self.inplanes != out:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, out, 1, stride, bias = False),
                nn.BatchNorm2d(out)
            )
        
        layers = [ResBlock(self.inplanes, out, stride, downsample)]
        self.inplanes = out
        for i in range(1, num_layer):
            layers.append(ResBlock(self.inplanes, out))
        
        return nn.Sequential(*layers)
        
        
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = F.pad(out, (1, 1, 1, 1), 'constant', 1)
        out = self.pool1(out)
        out = self.l1(out)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = self.avgpool(out)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out

In [10]:
resnet = ResNet()
resnet.to(device);

optimizer = torch.optim.Adam(resnet.parameters(), lr=0.05)
lossfn = nn.CrossEntropyLoss()

n_epochs = 25

In [11]:
%%time
train(resnet, n_epochs, trainloader, optimizer, lossfn)

0:0 || loss: 2.4809
0:97 || loss: 2.2780
0:194 || loss: 2.2701
0:291 || loss: 2.3027
0:388 || loss: 2.0652
Epoch done::0:391 || loss: 1.9919
1:485 || loss: 1.8793
1:582 || loss: 1.8794
1:679 || loss: 1.6374
1:776 || loss: 1.5290
Epoch done::1:782 || loss: 1.6774
2:873 || loss: 1.4741
2:970 || loss: 1.3968
2:1067 || loss: 1.4477
2:1164 || loss: 1.3819
Epoch done::2:1173 || loss: 1.3141
3:1261 || loss: 1.2904
3:1358 || loss: 1.2070
3:1455 || loss: 1.1553
3:1552 || loss: 1.2455
Epoch done::3:1564 || loss: 1.1016
4:1649 || loss: 1.0755
4:1746 || loss: 1.0902
4:1843 || loss: 0.9642
4:1940 || loss: 0.9839
Epoch done::4:1955 || loss: 1.1287
5:2037 || loss: 0.9791
5:2134 || loss: 1.1127
5:2231 || loss: 0.9039
5:2328 || loss: 0.8795
Epoch done::5:2346 || loss: 1.0680
6:2425 || loss: 0.7956
6:2522 || loss: 0.6960
6:2619 || loss: 0.7481
6:2716 || loss: 0.8749
Epoch done::6:2737 || loss: 0.7058
7:2813 || loss: 0.5954
7:2910 || loss: 0.5348
7:3007 || loss: 0.6832
7:3104 || loss: 0.5897
Epoch done::

In [12]:
print(evaluate(testloader, resnet))
print(evaluate(trainloader, resnet))

10000 8028
0.8028
50000 48477
0.96954


In [13]:
# torch.save(resnet.state_dict(), 'resnet.pt')